In [ ]:
%matplotlib inline

In [ ]:
import json
import numpy as np
import tensorflow as tf

In [ ]:
from statistics import median
from nltk.tokenize import word_tokenize

In [ ]:
def convert_json_to_review_and_rating(json_text):
    review_dict = json.loads(json_text)    
    return review_dict['reviewText'], review_dict['overall']

In [ ]:
def get_reviews_and_ratings(reviews_filepath):
    review_texts = list()
    ratings = list()
    with open(reviews_filepath) as reviews_file:
        for line in reviews_file:
            review_text, rating = convert_json_to_review_and_rating(line)
            review_texts.append(review_text)
            ratings.append(int(rating))
            
    return review_texts, ratings

In [ ]:
def texts_to_indexed_word_sequences(review_texts):
    word_indices = dict()
    indexed_sequences = list()
    word_index = 1
    
    for review_text in review_texts:
        tokens = word_tokenize(review_text)
        indexed_sequence = list()
        for token in tokens:
            if token not in word_indices:
                word_indices[token] = word_index
                indexed_sequence.append(word_index)
                word_index += 1
            else:
                indexed_sequence.append(word_indices[token])
        indexed_sequences.append(np.asarray(indexed_sequence))
        
    return word_indices, indexed_sequences

In [ ]:
reviews_filepath = "/home/v2john/datasets/amazon/reviews_electronics.json"

In [ ]:
review_texts, ratings = get_reviews_and_ratings(reviews_filepath)
print(len(review_texts), len(ratings))

In [ ]:
word_indices, indexed_sequences = texts_to_indexed_word_sequences(review_texts)

In [ ]:
VOCAB_SIZE = len(word_indices)
print("VOCAB_SIZE: ", VOCAB_SIZE)

EMBEDDING_SIZE = 300
print("EMBEDDING_SIZE: ", EMBEDDING_SIZE)

MAX_SEQUENCE_LENGTH = int(median([len(sequence) for sequence in indexed_sequences]))
print("MAX_SEQUENCE_LENGTH: ", MAX_SEQUENCE_LENGTH)

NUM_CLASSES = len(set(ratings))
print("NUM_CLASSES: ", NUM_CLASSES)

In [ ]:
def pad_indexed_sequences(indexed_sequences, max_sequence_length):
    new_indexed_sequences = list()
    for sequence in indexed_sequences:
        if len(sequence) >= max_sequence_length:
            new_indexed_sequences.append(sequence[:max_sequence_length])
        else:
            shortfall = max_sequence_length - len(sequence)
            new_indexed_sequences.append(
                np.pad(sequence, (0, shortfall), 'constant', 
                       constant_values=(0, 0)))
    return np.asarray(new_indexed_sequences)

def convert_labels_to_logits(ratings, num_classes):
    one_hot_ratings = list()
    for rating in ratings:
        one_hot_rating = np.zeros(num_classes)
        one_hot_rating[rating - 1] = 1
        one_hot_ratings.append(one_hot_rating)
        
    return np.asarray(one_hot_ratings)

def tensorize_sequences_and_labels(indexed_sequences, ratings, max_sequence_length, num_classes):
    return pad_indexed_sequences(indexed_sequences, max_sequence_length), \
        convert_labels_to_logits(ratings, num_classes)

In [ ]:
indexed_sequences, labels = tensorize_sequences_and_labels(
    indexed_sequences, ratings, MAX_SEQUENCE_LENGTH, NUM_CLASSES)

In [ ]:
indexed_sequences.shape, labels.shape

In [ ]:
graph_1 = tf.Graph()
with graph_1.as_default():
    
    input_x = tf.placeholder(
        tf.int32, [None, MAX_SEQUENCE_LENGTH], name="input_x")
    input_y = tf.placeholder(
        tf.int32, [None, NUM_CLASSES], name="input_y")

    word_embeddings = tf.get_variable(
        shape=[VOCAB_SIZE, EMBEDDING_SIZE], name="word_embeddings", 
        dtype=tf.float32)
    print("word_embeddings.shape: ", word_embeddings.shape)
    
    embedded_sequence = tf.nn.embedding_lookup(
        word_embeddings, input_x, name="embedded_sequence")
    print("embedded_sequence.shape: ", embedded_sequence.shape)
    
    conv_1 = tf.layers.conv1d(
        inputs=embedded_sequence, filters=300, kernel_size=3, name="conv_1")
    print("conv_1.shape: ", conv_1.shape)
    max_pool_1 = tf.layers.max_pooling1d(
        inputs=conv_1, pool_size=2, strides=2, name="max_pool_1")
    print("max_pool_1.shape: ", max_pool_1.shape)
    
    conv_2 = tf.layers.conv1d(
        inputs=max_pool_1, filters=150, kernel_size=3, name="conv_2")
    print("conv_2.shape: ", conv_2.shape)
    max_pool_2 = tf.layers.max_pooling1d(
        inputs=conv_2, pool_size=2, strides=2, name="max_pool_2")
    print("max_pool_2.shape: ", max_pool_2.shape)
    
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(128)
    states_series, current_state = tf.nn.dynamic_rnn(
        cell=lstm_cell_1, inputs=max_pool_2, dtype=tf.float32)
    print("current_state.h.shape: ", current_state.h.shape)
    
    dense_1 = tf.layers.dense(
        inputs=current_state.h, units=NUM_CLASSES, name="dense_1")
    print("dense_1.shape: ", dense_1.shape)
    
    softmax_output = tf.nn.softmax(dense_1, name="softmax")
    print("softmax_output.shape: ", softmax_output.shape)
    
    one_hot_label = tf.one_hot(
        indices=input_y-1, depth=1, on_value=1, off_value=0,
        name="one_hot_label")
    one_hot_label = tf.reshape(one_hot_label, tf.shape(softmax_output))
    print("one_hot_label.shape: ", one_hot_label.shape)
    
    loss = tf.losses.softmax_cross_entropy(
        one_hot_label, softmax_output)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [ ]:
epoch_reporting_interval = 50

with tf.Session(graph=graph_1) as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(0, 1000):
        _, l = \
            sess.run([optimizer, loss], 
                feed_dict={
                    input_x: indexed_sequences[i*100: (i+1)*100],
                    input_y: labels[i*100: (i+1)*100]
                }
            )
        
        if (i % epoch_reporting_interval == 0):
            print("Training epoch: ", i, ", Loss: ", l)
            
    final_predictions = \
        sess.run(
            [softmax_output], 
            feed_dict={
                input_x: indexed_sequences, 
                input_y: labels
            }
        )
        
    for prediction in final_predictions:
        print(sess.run(tf.contrib.seq2seq.hardmax(prediction)))